In [ ]:
import os
import pandas as pd
import datetime
import numpy as np

In [2]:

positive = pd.read_excel('/Users/a2/Downloads/Hackathon2/41591_2021_1593_MOESM3_ESM.xlsx')
positive = positive.drop('Device', axis=1)
df = positive.copy()

In [3]:
base_folder = '/Users/a2/Downloads/Hackathon2/Phase2_Data'
pid_folders = os.listdir(base_folder)
for f in pid_folders:
    if f not in positive['Participant ID'].values:
        os.system(f"rm -rf {base_folder}/{f}")

In [11]:
from pickletools import int4


def get_hr_st(id, date):
    files = os.listdir(f'/Users/a2/Downloads/Hackathon2/Phase2_Data/{id}')
    files.sort()
    apple = False
    if files[1].find("Non") != -1:
        apple  = True
    temp = pd.read_csv(f'/Users/a2/Downloads/Hackathon2/Phase2_Data/{id}/{files[0]}')
    heartrate = temp.copy()

    print(1)

    base = 	heartrate.iloc[0]['datetime']
    summe = 0
    anzahl = 0
    heartrate_scaled = pd.DataFrame()
    for n, d in enumerate(heartrate['datetime']):
        if str(d)[:-3] == base[:-3]:
            summe += heartrate['heartrate'][n]
            anzahl += 1
        else:
            temp = [[pd.to_datetime(f"{base[:-3]}:00"), int(summe/anzahl)]]
            base = d
            heartrate_scaled = heartrate_scaled.append(temp)
            summe = heartrate['heartrate'][n] 
            anzahl = 1
    
    print(2)

    heartrate_scaled.rename(columns={0:'datetime', 1:'heartrate', 2: 'steps'}, inplace=True)
    temp = pd.read_csv(f'/Users/a2/Downloads/Hackathon2/Phase2_Data/{id}/{files[1]}')
    steps = temp.copy()
    
    steps.rename({'end_datetime': 'datetime'})
    count = 0
    steps_normalized = pd.DataFrame()
    for i in range(len(steps)-1): 
        value = (steps.iloc[i]['steps']+ steps.iloc[i+1]['steps'] )/ ((pd.to_datetime(steps.iloc[i+1]['datetime']) - pd.to_datetime(steps.iloc[i]['datetime'])).total_seconds()/60)/2
        while pd.to_datetime(heartrate_scaled.iloc[count]['datetime']) < pd.to_datetime(steps.iloc[i+1]['datetime']) :
            steps_normalized = steps_normalized.append([value])
            count = count + 1
            try:
                assert(count<len(heartrate_scaled))
            except BaseException:
                break
    heartrate_scaled['steps'] = steps_normalized

    positive_np = np.zeros(heartrate_scaled.shape[0], dtype=int)

    print(3)

    positive = False

    for i in range(len(heartrate_scaled)):
        if date < heartrate_scaled.iloc[i]['datetime']:
            positive_np[i] = 1
    
    heartrate_scaled['positive'] = pd.DataFrame(positive_np)
    return heartrate_scaled
    # date ist 4 Tage zu spät
    

In [12]:
# 4 days 
for i in range(len(df)):
    # 
    # hl.to_csv()
    # print(df['Participant ID'][i])
    try:
        date = pd.to_datetime(df['COVID-19 Symptom Onset'][i])
    except BaseException:
        date = pd.to_datetime(df['COVID-19 Test Date'])[i] + pd.DateOffset(days=-4)
    hl = get_hr_st(df['Participant ID'][i], date)


1
2
3
1


KeyboardInterrupt: 